In [4]:
!pip install --upgrade tables
!pip install --upgrade pydot_ng
!pip install --upgrade pydot
!pip install --upgrade pydotplus
!pip install --upgrade graphviz

Requirement already up-to-date: tables in c:\users\abhishek\anaconda3\lib\site-packages (3.4.4)
Requirement already up-to-date: pydot_ng in c:\users\abhishek\anaconda3\lib\site-packages (2.0.0)
Requirement already up-to-date: pydot in c:\users\abhishek\anaconda3\lib\site-packages (1.4.0)
Requirement already up-to-date: pydotplus in c:\users\abhishek\anaconda3\lib\site-packages (2.0.2)
Requirement already up-to-date: graphviz in c:\users\abhishek\anaconda3\lib\site-packages (0.10.1)


In [5]:
import pandas as pd
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
import tensorflow.keras.backend as K

import math
from sklearn.preprocessing import RobustScaler
from collections import deque

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
#Path here

base_path = 'data/created/'
sp500_path = '/data/sp/^GSPC.csv'
created_data_path = base_path + 'data500.h5'
created_data_path_new = base_path + 'data500_new.h5'
technical500_path = base_path + 'technical500.h5'
news_path = '/data/news/'

In [0]:
import pandas as pd
#data = pd.read_hdf("/content/drive/My Drive/app/data2.h5", 'new_data');
data = pd.read_hdf(created_data_path_new);

# Import the CSV file with technical data and sentiment analysis

In [8]:
data = pd.read_hdf('data2.h5', 'new_data')
#data.index = pd.to_datetime(data.index, format='%Y-%m-%d')
data['Y'] = data.lrets.shift(-1)
data.dropna(axis=0, inplace=True)
data.head()

FileNotFoundError: File data2.h5 does not exist

In [9]:
corr = data.corr()
f, ax = plt.subplots(figsize=(5, 4))
plt.title("Correlation between variables")
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

NameError: name 'data' is not defined

# Train Test split for recurrent neural net

http://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html

https://pythonprogramming.net/crypto-rnn-model-deep-learning-python-tensorflow-keras/?completed=/balancing-rnn-data-deep-learning-python-tensorflow-keras/

In [10]:
d = deque('ghi')                 # make a new deque with three items
for elem in d:                   # iterate over the deque's elements
    print (elem.upper())

G
H
I


In [11]:
def preprocess_df(data, shuffle=True):
    mapped_data = data.copy()
    SEQ_LEN = 60

    scaler = RobustScaler()
    data = scaler.fit_transform(data)

    sequential_data = []  # this is a list that will CONTAIN the sequences
    prev_days = deque(maxlen=SEQ_LEN)  # These will be our actual sequences. They are made with deque, which keeps the maximum length by popping out older values as new ones come in

    for i in data:  # iterate over the values
        prev_days.append([n for n in i[:-1]])  # store all but the target

        # if prev_days have 60 days of data
        if len(prev_days) == SEQ_LEN:
            # sequential_data = [prev_days_data, target variable]
            sequential_data.append([np.array(prev_days), i[-1]])  
    if shuffle == True:
        random.shuffle(sequential_data)  # shuffle for good measure.

    X = []
    y = []

    for seq, target in sequential_data:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)
    
    if shuffle == True:
        return np.array(X), y 
    else:
        return np.array(X), y, mapped_data, scaler

In [12]:
times = sorted(data.index.values)
last_5pct = sorted(data.index.values)[-int(0.05*len(times))]

validation_main_df = data[(data.index >= last_5pct)]
main_df = data[(data.index < last_5pct)]

prescaler = RobustScaler()
prescaler = prescaler.fit(validation_main_df)

X_train, y_train = preprocess_df(main_df)
X_val, y_val, mapped_data, prescaler = preprocess_df(validation_main_df, shuffle=False)

NameError: name 'data' is not defined

# Stochastic Gradient descent with restart

Replacing Adam optimizer with SGDR for better performance

https://www.jeremyjordan.me/nn-learning-rate/

In [13]:
class SGDRScheduler(Callback):
    '''Cosine annealing learning rate scheduler with periodic restarts.
    # Usage
        ```python
            schedule = SGDRScheduler(min_lr=1e-5,
                                     max_lr=1e-2,
                                     steps_per_epoch=np.ceil(epoch_size/batch_size),
                                     lr_decay=0.9,
                                     cycle_length=5,
                                     mult_factor=1.5)
            model.fit(X_train, Y_train, epochs=100, callbacks=[schedule])
        ```
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        lr_decay: Reduce the max_lr after the completion of each cycle.
                  Ex. To reduce the max_lr by 20% after each cycle, set this value to 0.8.
        cycle_length: Initial number of epochs in a cycle.
        mult_factor: Scale epochs_to_restart after each full cycle completion.
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: http://arxiv.org/abs/1608.03983
    '''
    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

# Learning rate finder

In [14]:
class LRFinder(Callback):
    
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset. 
    
    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5, 
                                 max_lr=1e-2, 
                                 steps_per_epoch=np.ceil(epoch_size/batch_size), 
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])
            
            lr_finder.plot_loss()
        ```
    
    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`. 
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient. 
        
    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''
    
    def __init__(self, min_lr=1e-5, max_lr=1e-2, steps_per_epoch=None, epochs=None):
        super().__init__()
        
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations 
        return self.min_lr + (self.max_lr-self.min_lr) * x
        
    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
            
        K.set_value(self.model.optimizer.lr, self.clr())
 
    def plot_lr(self):
        '''Helper function to quickly inspect the learning rate schedule.'''
        plt.plot(self.history['iterations'], self.history['lr'])
        plt.yscale('log')
        plt.xlabel('Iteration')
        plt.ylabel('Learning rate')
        
    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')

# Build up GRU model

- Adam Optimizer = RMSProp + Momentum

- RMSE should be more useful when large errors are particularly undesirable.

- Batchnorm and dropout for regularization

- https://www.quora.com/What-is-the-difference-between-dropout-and-batch-normalization

In [15]:
def build_model():
    model = Sequential()
    model.add(CuDNNLSTM(256, input_shape=(X_train.shape[1:]), return_sequences=True))
    model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

    model.add(CuDNNLSTM(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    
    model.add(CuDNNLSTM(64))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(1, activation='tanh'))
    
    opt = tf.keras.optimizers.RMSprop(lr=7e-3)
    model.compile(loss='mse',optimizer=opt, metrics=['mse'])
    return model

In [16]:
model = build_model()

NameError: name 'X_train' is not defined

In [17]:
from keras.utils import plot_model

#later plot the model image
#plot_model(model, to_file='model.png')

Using TensorFlow backend.


In [18]:
# find learning rate first
epoch_size = 20
batch_size = 256

lr_finder = LRFinder(min_lr=1e-5, 
                     max_lr=1e-2, 
                     steps_per_epoch=np.ceil(epoch_size/batch_size), 
                     epochs=3)
model.fit(X_train, y_train, callbacks=[lr_finder])

lr_finder.plot_loss()

NameError: name 'model' is not defined

In [19]:
model = build_model()
schedule = SGDRScheduler(min_lr=1e-3,
                         max_lr=1e-2,
                         steps_per_epoch=np.ceil(epoch_size/batch_size),
                         lr_decay=0.9,
                         cycle_length= 3,
                         mult_factor=1.5)

model.fit(
    X_train,
    y_train,
    epochs=20, 
    callbacks=[schedule],
    validation_split=0.1,
    verbose=1)

NameError: name 'X_train' is not defined

In [20]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train RMSE: %2f' % math.sqrt(trainScore[0]))

testScore = model.evaluate(X_val, y_val, verbose=0)
print('Test RMSE: %2f'% math.sqrt(testScore[0]))

NameError: name 'model' is not defined

In [21]:
p = model.predict(X_val)

NameError: name 'model' is not defined

In [22]:
X_val.shape

NameError: name 'X_val' is not defined

In [24]:
p

NameError: name 'p' is not defined

In [25]:
p.shape

NameError: name 'p' is not defined

In [26]:
data.shape

NameError: name 'data' is not defined

In [27]:
#padding = np.zeros((1249, 15))
#padding.shape

In [28]:
#pred_result = pd.DataFrame(data=padding)

In [29]:
#pred_result.shape

In [30]:
#pred_result.keys()

In [31]:
#pred_result.tail

In [32]:
#pred_result[14]

In [33]:
data.tail()

NameError: name 'data' is not defined

If the chosen model fits worse than a horizontal line, then R2 is negative.

In [34]:
p.shape

NameError: name 'p' is not defined

In [35]:
y_val

NameError: name 'y_val' is not defined

In [36]:
#SP-500:
padding = np.zeros((265, 9))
pred_result = pd.DataFrame(data=padding)
pred_result['9'] = p
pred = prescaler.inverse_transform(pred_result)

val_result = pd.DataFrame(data=padding)
val_result['9'] = y_val
val = prescaler.inverse_transform(val_result)

NameError: name 'p' is not defined

# Plotting Predicted value vs True value

In [37]:
#Intrinio App model
#working code!!

#p is predicted values (3x1)
#y_val is actual values (3x1)

#padding = np.zeros((3, 9))

#Predicted Values
#pred_result = pd.DataFrame(data=padding)
#pred_result['9'] = p
#pred = prescaler.inverse_transform(pred_result)


#Actual Values
#val_result = pd.DataFrame(data=padding)
#val_result['9'] = y_val
#val = prescaler.inverse_transform(val_result)

In [38]:
pred = pred[:, -1]
val = val[:, -1]

NameError: name 'pred' is not defined

In [39]:
plt.figure(figsize=(21,7))
plt.plot(data.index[-len(pred):], np.expm1(pred)*100, color='red', label='Prediction')
plt.plot(data.index[-len(val):], np.expm1(val)*100, color='blue', label='True Value')
plt.xlabel("Date")
plt.ylabel("Return (%)")
plt.title('Predicted value vs True Value')
plt.legend(loc='best')
plt.show()

NameError: name 'data' is not defined

<Figure size 1512x504 with 0 Axes>

# The residual plot is not normal

In [40]:
import scipy.stats as stats
residual = np.expm1(pred) - np.expm1(val)
stats.probplot(residual, dist="norm", plot=plt)

NameError: name 'pred' is not defined

In [41]:
# Updates to be continued